This is official pytorch tutorial: <a href=https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py> Blitz Tutorial<a>

What is done in this tutorial:
    1. Load and normalize the CIFAR10 training and test datasets using torchvision
    2. Define a Convolutional Neural Network
    3. Define a loss function
    4. Train the network on the training data
    5. Test the network on the test data

TORCHVISION:
    The torchvision package consists of popular datasets, model architectures,
    and common image transformations for computer vision.

In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.functional import one_hot

import numpy as np
import h5py
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
embeddings = h5py.File("../../data/embeddings.h5", "r")
with open("../../data/seq_anno_hash.pickle", 'rb') as handle:
    proteins_and_hashes = pickle.load(handle)

In [5]:
label_prot = pd.read_csv("../data_splits/train_prot_id_labels.csv")

X_train, X_test = train_test_split(label_prot, test_size = 0.3, train_size=0.7, random_state=42, stratify=label_prot["label"])
print(len(X_train))
print(len(X_test))
X_train

3246
1392


,prot_id,label
3831,O16299,G
2899,P03524,SP_TM
2165,B6EU02,G
4334,Q95UE8,G_SP
3760,Q0VD86,G
...,...,...
3631,Q8BXQ2,SP_TM
1467,Q8I7Z8,G
576,P51946,G
433,Q12211,G


In [6]:
train_embeddings = list()
train_labels = list()
train_protein_ids = list()
for index, row in X_train.iterrows():
    hash_code = proteins_and_hashes[row["label"]][row["prot_id"]][2]
    mean_embedding = np.mean(embeddings.get(hash_code), axis=0)
    label = row["label"]
    id = row["prot_id"]
    train_protein_ids.append(id)
    train_embeddings.append(mean_embedding)
    train_labels.append(label)

test_embeddings = list()
test_labels = list()
test_protein_ids = list()
for index, row in X_test.iterrows():
    hash_code = proteins_and_hashes[row["label"]][row["prot_id"]][2]
    mean_embedding = np.mean(embeddings.get(hash_code), axis=0)
    label = row["label"]
    id = row["prot_id"]
    test_protein_ids.append(id)
    test_embeddings.append(mean_embedding)
    test_labels.append(label)

In [13]:
df

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,labels
0,-0.011368,-0.041199,0.004509,0.024597,0.006310,0.011063,-0.057220,-0.078491,0.002256,-0.007217,...,-0.009239,0.001315,-0.018234,0.030777,0.043732,-0.015915,-0.052979,0.040161,0.048889,G
1,-0.017212,-0.026138,0.082520,-0.003960,0.007973,0.053406,-0.015556,-0.048523,0.056610,0.052582,...,-0.009422,-0.006477,-0.013466,0.028137,-0.014320,-0.001544,-0.044952,0.050262,0.085754,SP_TM
2,-0.093140,-0.068115,-0.028992,0.065491,0.003998,0.018326,-0.051361,-0.077942,0.014542,-0.000029,...,-0.039612,-0.049194,0.011292,0.029770,0.032501,-0.025558,0.030136,-0.006653,0.068604,G
3,0.046814,0.033203,0.031403,0.009018,0.001868,0.009888,-0.074585,-0.052277,0.017136,-0.023804,...,-0.011879,0.000846,-0.009827,0.071533,0.014061,0.012444,0.044098,0.045319,0.014381,G_SP
4,0.016586,0.013885,0.028610,0.000306,0.017242,0.044983,-0.026459,-0.036957,0.064697,-0.024429,...,-0.013718,-0.033630,0.031128,0.047241,0.013695,-0.014023,0.012863,-0.007317,0.027237,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,0.026260,-0.013016,0.033203,-0.011421,0.005230,0.013847,-0.043030,-0.079651,0.033112,0.031113,...,0.027863,-0.007492,-0.075806,0.035034,0.022888,-0.008621,-0.001949,0.025421,0.054199,TM
1388,0.019928,-0.016663,0.018845,0.033356,0.008423,0.035187,0.010178,-0.074585,0.030396,0.044067,...,-0.039124,-0.008720,-0.030411,0.018265,0.064941,0.031647,0.006977,0.070618,0.019775,G_SP
1389,0.002785,0.002302,-0.010139,0.031113,-0.036102,0.043854,-0.084900,-0.040833,0.008842,-0.025436,...,-0.025436,-0.018982,-0.045197,0.049438,0.025604,-0.065735,-0.019043,-0.020721,0.025833,G
1390,0.004662,-0.009460,-0.027359,-0.012222,-0.012253,-0.004524,-0.063599,-0.046326,-0.025650,-0.029236,...,-0.037659,0.005707,-0.043182,0.023422,0.027954,0.006458,-0.044891,0.008850,0.041687,G


### Create dataloader

In [17]:
label_mappings = {
    'G_SP': 0,
     'G': 1,
     'SP_TM': 2,
     'TM': 3
}
reverse_label_mappings = {val: key for key, val in label_mappings.items()}

In [ ]:
reverse_label_mappings

In [ ]:
type(train_embeddings)

In [8]:
train_embeddings = torch.Tensor(np.array(train_embeddings))
train_labels = torch.Tensor([label_mappings[label] for label in  train_labels])

In [9]:
test_embeddings = torch.Tensor(np.array(test_embeddings))
test_labels = torch.Tensor([label_mappings[label] for label in  test_labels])

In [10]:
train_labels = one_hot(train_labels.to(torch.int64), 4)
dataset = TensorDataset(train_embeddings, train_labels)
dataloader = DataLoader(dataset, batch_size=4)

In [11]:
test_labels = one_hot(test_labels.to(torch.int64), 4)
test_dataset = TensorDataset(test_embeddings, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [12]:
import torch.nn as nn
import torch.nn.functional as F


class MLP(nn.Module):
    """ Define a model with two convolution layers each followed by a Max Pooling layer
    Then the embeddings are Flattened and forwarded into two FCls.
    The last layer is a FCL with 10 neurons.
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1024, 512)  # in channels -> 16 * 5 ^ 2 (kernel size 5)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 4)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        #x = F.softmax(x, dim=4)
        return x


net = MLP()

In [13]:
import torch.optim as optim

# The function is indicating the performance of the model.
# During the training process this function should be minimized
criterion = nn.CrossEntropyLoss()

# The minimization is achieved through Stochastic Gradient Descent
optimizer = optim.Adagrad(net.parameters(), lr=0.01)

In [14]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        """
        shape of inputs: torch.Size([4, 3, 32, 32])
            Batchsize: 4
            Channels: 3 (Red, Green, Blue)
            Image size: 32 x 32

        labels: tensor([9, 3, 0, 3])
            9: class of image 0 in batch
            3: class of image 1 in batch
            ...
        """
        inputs, labels = data

        """ zero the parameter gradients after every batch
        This is necessary because the gradients (directions of how the weigths and biases
        will be updated) are accumulated in each backward pass.
        https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        """
        optimizer.zero_grad()  # SGD

        # forward + backward + optimize
        # shape outputs: torch.Size([4, 10])
        # for every image a prediction
        #print(f"{inputs}")
        outputs = net(inputs)
        #print(f"{outputs} \t {labels}")

        # the first iteration CrossEntropy: tensor(2.3100, grad_fn=<NllLossBackward0>)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # running loss after 3 iterations: 6.894119024276733
        # Why is the loss added?
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {(running_loss / 99):.3f}')
            running_loss = 0.0

print('Finished Training')

[1, 100] loss: 0.567
[1, 200] loss: 0.303
[1, 300] loss: 0.273
[1, 400] loss: 0.182
[1, 500] loss: 0.178
[1, 600] loss: 0.119
[1, 700] loss: 0.123
[1, 800] loss: 0.158
[2, 100] loss: 0.056
[2, 200] loss: 0.080
[2, 300] loss: 0.103
[2, 400] loss: 0.072
[2, 500] loss: 0.096
[2, 600] loss: 0.054
[2, 700] loss: 0.072
[2, 800] loss: 0.065
[3, 100] loss: 0.032
[3, 200] loss: 0.045
[3, 300] loss: 0.063
[3, 400] loss: 0.038
[3, 500] loss: 0.056
[3, 600] loss: 0.031
[3, 700] loss: 0.034
[3, 800] loss: 0.022
[4, 100] loss: 0.022
[4, 200] loss: 0.017
[4, 300] loss: 0.034
[4, 400] loss: 0.027
[4, 500] loss: 0.042
[4, 600] loss: 0.024
[4, 700] loss: 0.021
[4, 800] loss: 0.015
[5, 100] loss: 0.018
[5, 200] loss: 0.010
[5, 300] loss: 0.017
[5, 400] loss: 0.015
[5, 500] loss: 0.021
[5, 600] loss: 0.014
[5, 700] loss: 0.015
[5, 800] loss: 0.006
Finished Training


### Evaluate the model on the test data
This could be done with TorchMetrics but we will do this manually here

In [15]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)

        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_labels)} test embeddings: {(100 * correct / total):.2f}')

Accuracy of the network on the 1392 test embeddings: 97.70


In [16]:
# prepare to count predictions for each class
classes = list(label_mappings.keys())
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        _, labels = torch.max(labels, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                         accuracy))

"""
Accuracy for class G_SP  is: 97.7 %
Accuracy for class G     is: 99.3 %
Accuracy for class SP_TM is: 79.4 %
Accuracy for class TM    is: 86.2 %
"""

Accuracy for class G_SP  is: 95.5 %
Accuracy for class G     is: 99.7 %
Accuracy for class SP_TM is: 91.7 %
Accuracy for class TM    is: 100.0 %


'\nAccuracy for class G_SP  is: 97.7 %\nAccuracy for class G     is: 99.3 %\nAccuracy for class SP_TM is: 79.4 %\nAccuracy for class TM    is: 86.2 %\n'

Create hashsum